In [267]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn import svm
from Utils import GeoUtils

from scipy import stats
from scipy.stats import norm, skew #for some statistics

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.kernel_ridge import KernelRidge
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from matplotlib.pyplot import figure
from sklearn.decomposition import PCA

%matplotlib inline

In [268]:
def getDistance(real, predict):
    lat1 = real[:,0]
    lon1 = real[:,1]
    lat2 = predict[:,0]
    lon2 = predict[:,1]

    return np.array(GeoUtils.GeoUtils.distanceInKm(lat1, lon1,lat2, lon2))

In [269]:
data = pd.read_csv("Dados/LocTreino_Equipe_4.csv")

bts_data = pd.read_csv("Dados/dados_BTSs.csv")

data.drop_duplicates(inplace=True)
data.shape
data = data.drop(['pontoId'], axis=1)
data['taBTS2'] = data['taBTS2'].fillna(data['taBTS2'].mean())
data['taBTS4'] = data['taBTS4'].fillna(data['taBTS4'].mean())

bts_data.describe()

,lat,lon,bcch,eirp
count,6.000000,6.000000,6.000000,6.000000e+00
mean,-8.071190,-34.895662,696.500000,5.559000e+01
std,0.005477,0.006370,54.617763,7.783606e-15
min,-8.076361,-34.908000,662.000000,5.559000e+01
25%,-8.075917,-34.894611,667.750000,5.559000e+01
50%,-8.072139,-34.894597,677.500000,5.559000e+01
75%,-8.066590,-34.893187,688.000000,5.559000e+01
max,-8.064583,-34.889444,806.000000,5.559000e+01


In [270]:
import math
d0 = 1	#must be greater than 0
PL = 2 #Pl expoent in free space
factor = 8 #For FPSL aproximate
B = 6 #number of BTSs
L = 3 #Dimension 2D
P = 1500 #number of points
GSM = 1800 #frequency for FPSL MHZ and meters (-27.55)https://en.wikipedia.org/wiki/Free-space_path_loss

def FPSL_estimate(vi):
	#FSPL(db), d in meters, f in MHZ , cte = -27.55
	#FSPL(db), d in meters, f in GHZ , cte = -87.55
	#FSPL(db), d in Kmeters, f in MHZ , cte = +32.45
	#FSPL(db), d in Kmeters, f in GHZ , cte = +92.45
	f = 1800 #original MHZ
	cte = +32.55
	#double sqrt seems to be the best one
	return pow((pow(10, (( float(vi)-cte)-(20*math.log(f,10)))/20)), 1./(factor*2))

In [271]:
x = data.drop(['lat', 'lon'], axis=1)

# x['pathPBTS1lon'] = x['pathBTS1']/bts_data['lon'][0]
# x['pathPBTS2lon'] = x['pathBTS2']/bts_data['lon'][1]
# x['pathPBTS3lon'] = x['pathBTS3']/bts_data['lon'][2]
# x['pathPBTS4lon'] = x['pathBTS4']/bts_data['lon'][3]
# x['pathPBTS5lon'] = x['pathBTS5']/bts_data['lon'][4]
# x['pathPBTS6lon'] = x['pathBTS6']/bts_data['lon'][5]



transformerX = StandardScaler().fit(x)
x_scaled = transformerX.transform(x)
x.describe()

,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0000
mean,116.654370,119.101165,123.863545,115.814052,121.320526,125.656192,0.663333,1.008733,2.525333,1.099388,1.093333,1.0560
std,15.229664,12.666188,9.031316,12.414620,12.552702,13.134365,0.698317,0.863886,0.967117,1.954747,0.856998,0.6692
min,86.256667,77.256667,88.340000,84.056667,79.523333,89.856667,0.000000,0.000000,1.000000,0.000000,0.000000,0.0000
25%,106.827500,111.673333,119.190000,106.583750,112.823333,116.077500,0.000000,0.000000,2.000000,0.000000,0.000000,1.0000
50%,117.156667,120.690000,124.273333,116.273333,122.840000,128.756667,1.000000,1.000000,3.000000,1.000000,1.000000,1.0000
75%,124.790000,128.808750,130.117083,125.590000,130.423333,135.996250,1.000000,1.000000,3.000000,1.000000,2.000000,2.0000
max,476.882449,144.890000,145.890000,145.423333,146.890000,150.123333,3.000000,11.090452,5.000000,38.747484,3.000000,2.0000


In [272]:
y = pd.DataFrame(columns=['distBTS1', 'distBTS2', 'distBTS3', 'distBTS4', 'distBTS5', 'distBTS6'])

bts1_coord = np.tile(bts_data[['lat', 'lon']].iloc[0].values, (1500,1))
y['distBTS1'] = getDistance(data[['lat', 'lon']].values, bts1_coord) 

bts2_coord = np.tile(bts_data[['lat', 'lon']].iloc[1].values, (1500,1))
y['distBTS2'] = getDistance(data[['lat', 'lon']].values, bts2_coord) 

bts3_coord = np.tile(bts_data[['lat', 'lon']].iloc[2].values, (1500,1))
y['distBTS3'] = getDistance(data[['lat', 'lon']].values, bts3_coord) 

bts4_coord = np.tile(bts_data[['lat', 'lon']].iloc[3].values, (1500,1))
y['distBTS4'] = getDistance(data[['lat', 'lon']].values, bts4_coord) 

bts5_coord = np.tile(bts_data[['lat', 'lon']].iloc[4].values, (1500,1))
y['distBTS5'] = getDistance(data[['lat', 'lon']].values, bts5_coord) 

bts6_coord = np.tile(bts_data[['lat', 'lon']].iloc[5].values, (1500,1))
y['distBTS6'] = getDistance(data[['lat', 'lon']].values, bts6_coord) 

y.head()

,distBTS1,distBTS2,distBTS3,distBTS4,distBTS5,distBTS6
0,0.879576,0.191390,1.294424,0.191390,1.291290,1.219177
1,0.665594,0.395403,1.309441,0.395403,1.103513,0.952860
2,0.629140,0.408747,1.339021,0.408747,1.067447,0.919427
3,1.026196,0.716060,0.883794,0.716060,1.471446,1.172785
4,0.793764,1.621548,2.642108,1.621548,0.390252,0.716741


In [273]:
transformerY = StandardScaler().fit(y)
y_scaled = transformerY.transform(y)

X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size = 0.3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

invX = transformerX.inverse_transform
invY = transformerY.inverse_transform

(1050, 12) (450, 12) (1050, 6) (450, 6)


In [274]:
knn = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3))
knn.fit(X_train, y_train)

knnreal = invY(y_test)
knnpredict = invY(knn.predict(X_test))

print (np.mean((knnpredict - knnreal)**2, axis=0))

print (np.std((knnpredict - knnreal)**2, axis=0))

[0.00115519 0.00147281 0.0018542  0.00147281 0.00171774 0.00121543]
[0.00300178 0.00624361 0.01391537 0.00624361 0.01356853 0.00322201]


In [250]:
kridge = MultiOutputRegressor(KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5))
kridge.fit(X_train, y_train)

kridgereal = invY(y_test)
kridgepredict = invY(kridge.predict(X_test))

#print(getDistance(kridgereal, kridgepredict).mean())
print (np.mean((kridgereal - kridgepredict)**2, axis=0))

[0.00338556 0.0062881  0.0049994  0.0062881  0.00415078 0.00445343]


In [251]:
gboost = MultiOutputRegressor(GradientBoostingRegressor(n_estimators=1000, learning_rate=0.05,
                                   max_depth=50, max_features='sqrt',
                                   min_samples_leaf=5, min_samples_split=10, 
                                   loss='huber', random_state =5))
gboost.fit(X_train, y_train)

gboostreal = invY(y_test)
gboostpredict = invY(gboost.predict(X_test))

print (np.mean((gboostreal - gboostpredict)**2, axis=0))

[0.00180336 0.00277974 0.00253356 0.00271754 0.0019584  0.00233338]


In [252]:
svr = svm.SVR()
regr = MultiOutputRegressor(svr)
regr.fit(X_train,y_train)

svmreal = invY(y_test)
svmpredict = invY(regr.predict(X_test))

print (np.mean((svmreal - svmpredict)**2, axis=0))

[0.00179059 0.00223497 0.00284587 0.00223586 0.0023482  0.00230158]


In [253]:
lasso = MultiOutputRegressor(Lasso(alpha =0.0005, random_state=1))
lasso.fit(X_train, y_train)

lassoreal = invY(y_test)
lassopredict = invY(lasso.predict(X_test))

print (np.mean((lassoreal - lassopredict)**2, axis=0))

[0.00651711 0.0197366  0.01139312 0.0197366  0.0082497  0.00968674]


In [275]:
bts1_coord = np.tile(bts_data[['lat', 'lon']].iloc[0].values, (1500,1))
data['distBTS1'] = getDistance(data[['lat', 'lon']].values, bts1_coord) 

bts2_coord = np.tile(bts_data[['lat', 'lon']].iloc[1].values, (1500,1))
data['distBTS2'] = getDistance(data[['lat', 'lon']].values, bts2_coord) 

bts3_coord = np.tile(bts_data[['lat', 'lon']].iloc[2].values, (1500,1))
data['distBTS3'] = getDistance(data[['lat', 'lon']].values, bts3_coord) 

bts4_coord = np.tile(bts_data[['lat', 'lon']].iloc[3].values, (1500,1))
data['distBTS4'] = getDistance(data[['lat', 'lon']].values, bts4_coord) 

bts5_coord = np.tile(bts_data[['lat', 'lon']].iloc[4].values, (1500,1))
data['distBTS5'] = getDistance(data[['lat', 'lon']].values, bts5_coord) 

bts6_coord = np.tile(bts_data[['lat', 'lon']].iloc[5].values, (1500,1))
data['distBTS6'] = getDistance(data[['lat', 'lon']].values, bts6_coord) 

x2 = data.drop(['lat', 'lon'], axis=1)

# x2['pathPBTS1'] = x2['pathBTS1']/bts_data['bcch'][0]
# x2['pathPBTS2'] = x2['pathBTS2']/bts_data['bcch'][1]
# x2['pathPBTS3'] = x2['pathBTS3']/bts_data['bcch'][2]
# x2['pathPBTS4'] = x2['pathBTS4']/bts_data['bcch'][3]
# x2['pathPBTS5'] = x2['pathBTS5']/bts_data['bcch'][4]
# x2['pathPBTS6'] = x2['pathBTS6']/bts_data['bcch'][5]

# x2['pathPBTS1lat'] = x2['pathBTS1']/bts_data['lat'][0]
# x2['pathPBTS2lat'] = x2['pathBTS2']/bts_data['lat'][1]
# x2['pathPBTS3lat'] = x2['pathBTS3']/bts_data['lat'][2]
# x2['pathPBTS4lat'] = x2['pathBTS4']/bts_data['lat'][3]
# x2['pathPBTS5lat'] = x2['pathBTS5']/bts_data['lat'][4]
# x2['pathPBTS6lat'] = x2['pathBTS6']/bts_data['lat'][5]

# x2['pathPBTS1lon'] = x2['pathBTS1']/bts_data['lon'][0]
# x2['pathPBTS2lon'] = x2['pathBTS2']/bts_data['lon'][1]
# x2['pathPBTS3lon'] = x2['pathBTS3']/bts_data['lon'][2]
# x2['pathPBTS4lon'] = x2['pathBTS4']/bts_data['lon'][3]
# x2['pathPBTS5lon'] = x2['pathBTS5']/bts_data['lon'][4]
# x2['pathPBTS6lon'] = x2['pathBTS6']/bts_data['lon'][5]

# transformerX2 = StandardScaler().fit(x2)
# x_scaled2 = transformerX2.transform(x2)
x2.describe()

,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6,distBTS1,distBTS2,distBTS3,distBTS4,distBTS5,distBTS6
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
mean,116.654370,119.101165,123.863545,115.814052,121.320526,125.656192,0.663333,1.008733,2.525333,1.099388,1.093333,1.0560,0.651671,0.824295,1.673787,0.824295,0.894737,0.836718
std,15.229664,12.666188,9.031316,12.414620,12.552702,13.134365,0.698317,0.863886,0.967117,1.954747,0.856998,0.6692,0.338396,0.429586,0.489971,0.429586,0.451967,0.348651
min,86.256667,77.256667,88.340000,84.056667,79.523333,89.856667,0.000000,0.000000,1.000000,0.000000,0.000000,0.0000,0.054158,0.009910,0.870120,0.009910,0.026198,0.031430
25%,106.827500,111.673333,119.190000,106.583750,112.823333,116.077500,0.000000,0.000000,2.000000,0.000000,0.000000,1.0000,0.372980,0.527395,1.314656,0.527395,0.476412,0.591253
50%,117.156667,120.690000,124.273333,116.273333,122.840000,128.756667,1.000000,1.000000,3.000000,1.000000,1.000000,1.0000,0.603009,0.760281,1.669549,0.760281,0.858269,0.811690
75%,124.790000,128.808750,130.117083,125.590000,130.423333,135.996250,1.000000,1.000000,3.000000,1.000000,2.000000,2.0000,0.901601,1.023290,1.989360,1.023290,1.289275,1.097895
max,476.882449,144.890000,145.890000,145.423333,146.890000,150.123333,3.000000,11.090452,5.000000,38.747484,3.000000,2.0000,1.847645,2.274950,2.835246,2.274950,2.053067,1.527323


In [276]:
y2 = data[['lat', 'lon']]
y2.describe()

,lat,lon
count,1500.000000,1500.000000
mean,-8.070097,-34.894814
std,0.004232,0.004354
min,-8.077570,-34.906830
25%,-8.073197,-34.897387
50%,-8.069940,-34.895346
75%,-8.067113,-34.891169
max,-8.059339,-34.885067


In [277]:
# transformerY2 = StandardScaler().fit(y2)
# y2_scaled = transformerY2.transform(y2)

X_train2, X_test2, y_train2, y_test2 = train_test_split(x2, y2, test_size = 0.3)
print(X_train2.shape, X_test2.shape, y_train2.shape, y_test2.shape)

# invX2 = transformerX2.inverse_transform
# invY2 = transformerY2.inverse_transform
X_test2.head()

(1050, 18) (450, 18) (1050, 2) (450, 2)


,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6,distBTS1,distBTS2,distBTS3,distBTS4,distBTS5,distBTS6
1224,103.090000,116.656667,112.190000,101.056667,122.456667,120.590000,0,1.0,2,1.0,1,0,0.373437,0.793996,1.553350,0.793996,0.796905,0.503247
393,108.356667,127.790000,125.790000,100.723333,97.090000,130.890000,0,0.0,3,0.0,1,1,0.369340,0.546015,1.825921,0.546015,0.690854,0.826910
70,108.723333,125.256667,127.356667,98.123333,93.523333,128.756667,0,0.0,3,0.0,1,1,0.433960,0.478569,1.790209,0.478569,0.760076,0.886011
197,118.940000,128.090000,127.240000,110.115000,98.865000,128.015000,0,1.0,3,1.0,0,1,0.248170,0.996079,2.142461,0.996079,0.239934,0.577700
978,122.323333,129.590000,115.823333,125.990000,133.523333,126.656667,1,1.0,1,1.0,2,1,0.931522,0.924893,1.021218,0.924893,1.355530,0.949950


In [278]:
x_aux = pd.DataFrame()
x_aux = X_test2.drop(['distBTS1', 'distBTS2', 'distBTS3', 'distBTS4', 'distBTS5', 'distBTS6'], axis=1)
predicts = invY(knn.predict(transformerX.transform(x_aux.values)))
X_test2[['distBTS1', 'distBTS2', 'distBTS3', 'distBTS4', 'distBTS5', 'distBTS6']] = predicts
X_test2.head()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6,distBTS1,distBTS2,distBTS3,distBTS4,distBTS5,distBTS6
1224,103.090000,116.656667,112.190000,101.056667,122.456667,120.590000,0,1.0,2,1.0,1,0,0.354728,0.772229,1.563443,0.772229,0.783854,0.513836
393,108.356667,127.790000,125.790000,100.723333,97.090000,130.890000,0,0.0,3,0.0,1,1,0.390677,0.522973,1.813016,0.522973,0.714335,0.846282
70,108.723333,125.256667,127.356667,98.123333,93.523333,128.756667,0,0.0,3,0.0,1,1,0.390677,0.522973,1.813016,0.522973,0.714335,0.846282
197,118.940000,128.090000,127.240000,110.115000,98.865000,128.015000,0,1.0,3,1.0,0,1,0.295228,0.992310,2.169379,0.992310,0.257630,0.630664
978,122.323333,129.590000,115.823333,125.990000,133.523333,126.656667,1,1.0,1,1.0,2,1,0.920663,0.941623,1.044055,0.941623,1.341108,0.926872


In [279]:
pd.DataFrame(X_test2).describe()

,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6,distBTS1,distBTS2,distBTS3,distBTS4,distBTS5,distBTS6
count,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000
mean,117.031434,118.438289,124.177108,116.160237,121.795281,126.386687,0.715556,1.055776,2.526667,1.117217,1.148889,1.077778,0.681985,0.849658,1.669322,0.849658,0.920342,0.857445
std,11.892471,13.363960,9.264645,12.861599,13.155634,12.483828,0.733529,1.007121,1.021135,1.988659,0.889213,0.657792,0.359759,0.460724,0.504285,0.460724,0.472261,0.343566
min,86.790000,77.256667,88.340000,84.056667,80.790000,91.323333,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.078255,0.028654,0.873040,0.028654,0.047493,0.054000
25%,107.356667,110.090000,119.302500,106.465000,112.865000,117.240000,0.000000,0.000000,2.000000,0.000000,0.000000,1.000000,0.379146,0.513961,1.318010,0.513961,0.483257,0.625156
50%,117.556667,120.990000,124.623333,117.148333,124.365000,128.773333,1.000000,1.000000,3.000000,1.000000,1.000000,1.000000,0.637056,0.770220,1.678810,0.770220,0.870192,0.840683
75%,124.890000,128.464554,130.865000,126.481667,131.196250,135.915000,1.000000,1.000000,3.000000,1.000000,2.000000,2.000000,0.934117,1.021407,1.982277,1.021407,1.341101,1.119594
max,142.290000,141.890000,145.890000,145.423333,146.890000,150.123333,3.000000,11.090452,5.000000,38.747484,3.000000,2.000000,1.843604,2.271433,2.823232,2.271433,2.048966,1.524007


In [280]:
pd.DataFrame(X_train2).describe()

,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6,distBTS1,distBTS2,distBTS3,distBTS4,distBTS5,distBTS6
count,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000
mean,116.492771,119.385254,123.729160,115.665687,121.117060,125.343122,0.640952,0.988571,2.524762,1.091748,1.069524,1.046667,0.638728,0.814146,1.675941,0.814146,0.883543,0.828076
std,16.456552,12.350776,8.930562,12.221311,12.286083,13.397100,0.681807,0.794321,0.943522,1.940939,0.842125,0.674126,0.329186,0.414592,0.482980,0.414592,0.444167,0.350252
min,86.256667,78.365000,88.790000,84.656667,79.523333,89.856667,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.054158,0.009910,0.870120,0.009910,0.026198,0.033549
25%,106.248333,112.148333,119.131667,106.721250,112.840000,115.790000,0.000000,0.000000,2.000000,0.000000,0.000000,1.000000,0.369281,0.534809,1.320004,0.534809,0.472080,0.586767
50%,116.956667,120.573333,124.156667,116.106667,122.506667,128.740000,1.000000,1.000000,3.000000,1.000000,1.000000,1.000000,0.590734,0.749162,1.669549,0.749162,0.848318,0.793976
75%,124.752500,129.031667,129.735833,125.231667,129.965000,136.021250,1.000000,1.000000,3.000000,1.000000,2.000000,2.000000,0.882848,1.013571,1.989499,1.013571,1.245820,1.094274
max,476.882449,144.890000,144.790000,145.023333,146.090000,149.790000,3.000000,4.000000,5.000000,37.155187,3.000000,2.000000,1.843343,2.271313,2.835246,2.271313,2.048641,1.527323


In [281]:
trans_X = StandardScaler().fit(x2)
X_train2 = trans_X.transform(X_train2)
X_test2 = trans_X.transform(X_test2)

trans_Y = StandardScaler().fit(y2)

y_train2 = trans_Y.transform(y_train2)
y_test2 = trans_Y.transform(y_test2)

invY2 = trans_Y.inverse_transform
invX2 = trans_X.inverse_transform

knn2 = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3))
knn2.fit(X_train2, y_train2)

knn2real = invY2(y_test2)
knn2predict = invY2(knn2.predict(X_test2))

print(getDistance(knn2real, knn2predict).mean())

0.039301821324910245


In [261]:
gboost = MultiOutputRegressor(GradientBoostingRegressor(n_estimators=1000, learning_rate=0.05,
                                   max_depth=50, max_features='sqrt',
                                   min_samples_leaf=5, min_samples_split=10, 
                                   loss='huber', random_state =5))
gboost.fit(X_train2, y_train2)

gboostreal = invY2(y_test2)
gboostpredict = invY2(gboost.predict(X_test2))

print(getDistance(gboostreal, gboostpredict).mean())

0.0421667419176673


In [282]:
svr = svm.SVR()
regr = MultiOutputRegressor(svr)
regr.fit(X_train2,y_train2)

svmreal = invY2(y_test2)
svmpredict = invY2(regr.predict(X_test2))

print(getDistance(svmreal, svmpredict).mean())

0.04847397474510075


In [263]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
multioutputregressor = MultiOutputRegressor(model_xgb).fit(X_train2, y_train2)

realoutp = invY2(multioutputregressor.predict(X_test2))
realoutr = invY2(y_test2)                                         

print(getDistance(realoutr, realoutp).mean())

0.04927210168371669


In [283]:
kridge = MultiOutputRegressor(KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5))
kridge.fit(X_train2, y_train2)

kridgereal = invY2(y_test2)
kridgepredict = invY2(kridge.predict(X_test2))

print(getDistance(kridgereal, kridgepredict).mean())

0.03415497480639329


In [284]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [ ]:
knn = KNeighborsRegressor(n_neighbors=3)
knn2 = KNeighborsRegressor(n_neighbors=3)

gboost = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.05,
                                   max_depth=50, max_features='sqrt',
                                   min_samples_leaf=5, min_samples_split=10, 
                                   loss='huber', random_state =5)
svr = svm.SVR()
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
gboost2 = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=5, min_samples_split=10, 
                                   loss='huber', random_state =5)

gboost3 = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=5, min_samples_split=10, 
                                   loss='huber', random_state =5)

lasso = Lasso(alpha =0.0005, random_state=1)

stack = StackingAveragedModels(base_models = (gboost, svr, knn), meta_model=lasso)

stack2 = MultiOutputRegressor(StackingAveragedModels(base_models = (stack, knn2, gboost2), meta_model=gboost3))

stack2.fit(X_train2, y_train2)

stackreal = invY2(y_test2)
stackpredict = invY2(stack2.predict(X_test2))

print(getDistance(stackreal, stackpredict).mean())